# Exploring Hotels near a famous Motel during Vacation

People have come to the favorite vacation place in  the country - Coimbatore which is a city in southern INDIA.

People want to explore for hotels near a motel that they are staying

So I've got the information of all the hotels near the motel using the foursquare API and used it to construct the map using FOLIUM MAPS

# DATA SOURCE

The data used in the project is from Foursquare location Database. The data is obtained using the foursquare developer Client ID and secret key and the map is constructed

Importing libraries

In [8]:
import folium
from geopy.geocoders import Nominatim 
import pandas as pd
import numpy as np
import requests 
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 
print('Done')



Done


Setting up my forusquare API

In [17]:
CLIENT_ID = '0J0XCDTF30YW4JEWECV3YRA02GMCMULSVSE2BCULKVDUGB4Y'
CLIENT_SECRET = 'O4EWEIYTRVLRLQ2UG4U3NNOWXJ3SD3HOG2E03QDKHFAVSPA0'
LIMIT =30
VERSION='20180604'

Setting up my Location

In [18]:
city = 'Coimbatore'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

11.0018115 76.9628425


Searching for Hotels

In [28]:
search_query = 'Hotel'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results = requests.get(url).json()
#print(results['response']['venues'])

Converting into Pandas dataframe

In [29]:
locations=results['response']['venues']
df=json_normalize(locations)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bd1bbcf5e0cce726de8a184,"56, Sivasamy Road, Ram Nagar",IN,Coimbatore,India,NaN,1049,"[56, Sivasamy Road, Ram Nagar, Coimbatore 6410...","[{'label': 'display', 'lat': 11.01046346047451...",11.010463,76.966666,641009,Tamil Nadu,Hotel City Tower,v-1572674497
1,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,5393e7b7498e6a4f589039e7,NaN,IN,NaN,India,NaN,394,[India],"[{'label': 'display', 'lat': 11.000498, 'lng':...",11.000498,76.966198,NaN,NaN,Hotel Top in town,v-1572674497
2,[],False,51cc01fe498eaace9e568ec6,NaN,IN,Coimbatore,India,NaN,852,"[Coimbatore 641114, Tamil Nadu, India]","[{'label': 'display', 'lat': 10.99717395324831...",10.997174,76.969051,641114,Tamil Nadu,Hotel Naveen,v-1572674497
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bdf2d1f7ea362b5656043c4,No. 355 Variety Hall Road,IN,Coimbatore,India,NaN,406,"[No. 355 Variety Hall Road, Coimbatore, Tamil ...","[{'label': 'display', 'lat': 10.99837980131243...",10.998380,76.964106,NaN,Tamil Nadu,Hotel Rathna Residency Coimbatore,v-1572674497
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,5d9a248fded319000857fc4e,NaN,IN,Coimbatore,India,NaN,944,"[Coimbatore 641002, Tamil Nadu, India]","[{'label': 'display', 'lat': 11.006304, 'lng':...",11.006304,76.955510,641002,Tamil Nadu,Hotel Chola,v-1572674497


Cleaning data

In [78]:
clean_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')]+ ['id']
df2 = df.loc[:,clean_columns]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

df2['categories'] = df2.apply(get_category_type, axis=1)
df2.columns = [column.split('.')[-1] for column in df2.columns]

#dropping unnecessary cols
df22= df2.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
#dropping values with NA
df23 = df22.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

#only hotels and motels categories
items= ['Hotel', 'Motel']
df24=df23.loc[df23['categories'].isin(items)]

#replacing pincode 18 to standard 641018 value
df24=df24.replace(to_replace ="18",  
                            value ="641018") 


#choosing Motel Hem as my location
#choosing the hotel that has same pincode as 'Hotel Hem (Motal)'
resultdf=df24[df24.postalCode == '641018']
resultdf

,name,categories,address,lat,lng,postalCode,state
7,Hotel Park Plaza,Hotel,"90 / 62, Geetha Hall Road",10.998027,76.969980,641018,Tamil Nadu
17,Hotel Hem,Motel,"21,Geetha Hall Road",10.996965,76.968611,641018,Tamil Nadu


# MAP to visualize hotels near my location(Hotel Park Plaza)

In [79]:
map=folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(resultdf['lat'], resultdf['lng'], 
                                           resultdf['name'], resultdf['categories'],\
                                               resultdf['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

gitHub does not support displaying the map.
Kindly find the map image file at my repository titled 'HotelMap'
Thanks in advance.